# Setup

In [1]:
import sys
sys.path.append("../")

import random
import h5py
import copy
import os

import torch
import torch.nn as nn

from torchvision.models import *
from torchvision.transforms import *
from torch.utils.data import DataLoader, Dataset
from torch.optim import *
from torch.optim.lr_scheduler import *

import torch_pruning as tp

import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

from Dataset.dataset import *
from Dataset.data_generation import ArrhythmiaLabels
from Utils.classification import *
from Utils.model_loading import *

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using {device}")

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

d:\Anaconda3\envs\machine_learning\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using cuda:0


# Pruning

In [2]:
image_size = 152
dataloader = build_dataloader(
    train_path = "Data/mitbih_mif_train_small.h5",
    test_path  = "Data/mitbih_mif_test.h5",
    batch_size = 32,
    transform  = Resize((image_size, image_size), antialias=None),
)

model_name      = "vgg16_bn_custom"
base_model_path = "Pretrained/vgg16_bn_custom_ecg_ep50_i152.pth"
num_classes     = ArrhythmiaLabels.size
base_model      = load_model_from_pretrained(model_name, base_model_path, num_classes)

In [7]:
# Copy base model
pruned_model = copy.deepcopy(base_model).to("cpu")
pruned_model.eval()

# Get dummy input for tracing
dummy_input = torch.rand((1, 3, image_size, image_size))

# Ignore the last classification layer
ignored_layers = [pruned_model.classifier[6]]

# Pruning objects
imp = tp.importance.MagnitudeImportance(p=2)

pruner = tp.pruner.MagnitudePruner(
    model             = pruned_model,
    example_inputs    = dummy_input,
    importance        = imp,
    global_pruning    = False,
    pruning_ratio     = 0.8,
    max_pruning_ratio = 1.0,
    ignored_layers    = ignored_layers,
)

pruner.step()

pruned_model_stats = benchmark_model(pruned_model, dataloader["test"], name="Pruned")
display_model_stats(pruned_model_stats)

[I]: Benchmarking model Pruned
[I]: 	Getting model params and MACs
[I]: 	Measuring latency
[I]: 	Evaluating


[I]: Benchmarking for Pruned finished
Name:     Pruned
Accuracy: 82.76%
Latency:  7.0 ms
Params:   3 M
MACs:     284 M


In [8]:
# Finetuning
num_finetune_epochs = 30
pruned_model.to(device)
optimizer = SGD(pruned_model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)
scheduler = CosineAnnealingLR(optimizer, num_finetune_epochs)
criterion = nn.CrossEntropyLoss()

best_model_checkpoint = dict()
best_accuracy         = 0

print("Finetuning")
for epoch in range(num_finetune_epochs):
    train(pruned_model, dataloader["train"], criterion, optimizer, scheduler)
    accuracy = evaluate(pruned_model, dataloader["test"])

    if accuracy > best_accuracy:
        best_model_checkpoint["state_dict"] = copy.deepcopy(pruned_model.state_dict())
        best_accuracy = accuracy
    
    print(f"Epoch {epoch+1} Accuracy {accuracy:.2f}% / Best Accuracy: {best_accuracy:.2f}%")

# Save best model
if not os.path.exists("Pretrained/VGG-Pruned"):
    os.makedirs("Pretrained/VGG-Pruned")
    
save_path = f"Pretrained/VGG-Pruned/{model_name}_ecg_ep{num_finetune_epochs}_i{image_size}_p0.8.pth"
torch.save(best_model_checkpoint["state_dict"], save_path)

Finetuning


Epoch 1 Accuracy 95.15% / Best Accuracy: 95.15%


Epoch 2 Accuracy 97.71% / Best Accuracy: 97.71%


Epoch 3 Accuracy 87.70% / Best Accuracy: 97.71%


Epoch 4 Accuracy 97.27% / Best Accuracy: 97.71%


...

Epoch 26 Accuracy 97.58% / Best Accuracy: 98.26%


Epoch 27 Accuracy 98.14% / Best Accuracy: 98.26%


Epoch 28 Accuracy 97.31% / Best Accuracy: 98.26%


Epoch 29 Accuracy 97.59% / Best Accuracy: 98.26%


Epoch 30 Accuracy 96.86% / Best Accuracy: 98.26%


In [ ]:
ratios = [0.5, 0.6, 0.7, 0.8, 0.9]
for ratio in ratios:

    if ratio == 0.9:
        max_prune = 0.95
    else:
        max_prune = 0.9

    # Copy base model
    pruned_model = copy.deepcopy(base_model).to("cpu")
    pruned_model.eval()

    # Get dummy input for tracing
    dummy_input = torch.rand((1, 3, image_size, image_size))

    # Ignore the last classification layer
    ignored_layers = [pruned_model.classifier[6]]

    # Pruning objects
    imp = tp.importance.MagnitudeImportance(p=2)

    pruner = tp.pruner.MagnitudePruner(
        model             = pruned_model,
        example_inputs    = dummy_input,
        importance        = imp,
        global_pruning    = True,
        pruning_ratio     = ratio,
        max_pruning_ratio = max_prune,
        ignored_layers    = ignored_layers,
    )

    pruner.step()

    # Finetuning
    num_finetune_epochs = 20
    pruned_model.to(device)
    optimizer = SGD(pruned_model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)
    scheduler = CosineAnnealingLR(optimizer, num_finetune_epochs)
    criterion = nn.CrossEntropyLoss()

    best_model    = copy.deepcopy(pruned_model)
    best_accuracy = 0

    print("Finetuning")
    for epoch in range(num_finetune_epochs):
        train(pruned_model, dataloader["train"], criterion, optimizer, scheduler)
        accuracy = evaluate(pruned_model, dataloader["test"])

        if accuracy > best_accuracy:
            best_model = copy.deepcopy(pruned_model)
            best_accuracy = accuracy
        
        print(f"Epoch {epoch+1} Accuracy {accuracy:.2f}% / Best Accuracy: {best_accuracy:.2f}%")

    # Save best model
    if not os.path.exists("Pretrained/VGG-Pruned"):
        os.makedirs("Pretrained/VGG-Pruned")

    save_path = f"Pretrained/VGG-Pruned/{model_name}_ecg_ep{num_finetune_epochs}_i{image_size}_g{ratio}.pth"
    best_model.zero_grad()
    torch.save(best_model, save_path)